# <B> Setup for Nvidai NeMo with SageMaker </B>
* Container: codna_python3

## AutoReload

In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0. Install packages

In [ ]:
install_needed = True  # should only be True once
# install_needed = False

In [ ]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U datasets transformers
    !{sys.executable} -m pip install -U wget omegaconf text-unidecode sox
    
    ## Install NeMo
    !sudo yum install sox -y
    !sudo yum install libsndfile
    !pip install --upgrade --force-reinstall llvmlite
    BRANCH = 'main'
    !{sys.executable} -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
    
    IPython.Application.instance().kernel.do_shutdown(True)

## 1. Set roles

In [53]:
from sagemaker import get_execution_role

In [54]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


## 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **EC2ContainerRegistry**: "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess" <BR>
> **S3**: "arn:aws:iam::aws:policy/AmazonS3FullAccess"

## 2. Set default parameters

In [55]:
import boto3
import sagemaker

### Bucket / Prefix 설정

In [80]:
strRegionName = boto3.Session().region_name
strAccountId = boto3.client("sts").get_caller_identity().get("Account")
bucket_name = ' ' #'  ' # <-- 사용할 bucket 명을 추가해 주세요. ex) sagemaker-us-east-1-123456789123
prefix = ' '#'  ' ## <-- 작업할 prefix 명을 추가해 주세요. ex) nemo-test

## 3. Create training custom docker image

* docker build

In [6]:
!pygmentize custom-docker/Dockerfile

## docker build -f Dockerfile -t 322537213286.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.13.1-gpu-py39-nemo-main .

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.13.1-gpu-py39

RUN apt -y update && apt -y install sox
RUN apt-get -y install libsox-fmt-all

RUN pip install --no-cache-dir --upgrade pip \
 && pip install --no-cache-dir -U omegaconf hydra-core librosa sentencepiece youtokentome inflect sox \
 && pip install --no-cache-dir -U braceexpand webdataset editdistance jiwer jsonlines \
 && pip install --no-cache-dir -U pytorch-lightning \ 
 && pip install --no-cache-dir -qq https://github.com/pyannote/pyannote-audio/archive/develop.zip \ 
 && pip install --no-cache-dir git+https://github.com/huggingface/transformers \ 
 && pip install --no-cache-dir git+https://github.com/NVIDIA/NeMo.git@main
 
WORKDIR /


* Base 이미지의 region, account-id 확인 후 아래 파라미터 입력

### 1) AWS CLI 를 이용한 방식

In [ ]:
%%bash
strRepositoryName="nemo-test-training"  ## <-- 원하는 docker repostory 이름을 추가
strDockerDir="./custom-docker/"
strTag="latest"

cd ${strDockerDir}
echo $(pwd)
container_name=${strRepositoryName}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
# region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:${strTag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${container_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${container_name}" > /dev/null
fi

# # Get the login command from ECR and execute it directly
# $(aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin "763104351884.dkr.ecr.us-west-2.amazonaws.com")

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build -f Dockerfile -t ${fullname} .
# docker tag ${container_name} ${fullname}

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)
docker push ${fullname}
echo ${fullname}

In [75]:
strEcrRepositoryUri = ' '#' '  ## <-- 생성된 ECR의 URI를 넣어주세요.. ex) 123456789123.dkr.ecr.us-west-2.amazonaws.com/nemo-test:latest
#strEcrRepositoryUri = '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-training'#' '  ## <-- 생성된 ECR의 URI를 넣어주세요.. ex) 123456789123.dkr.ecr.us-west-2.amazonaws.com/nemo-test:latest

### 2) AWS BOTO3 SDK를 이용한 방식

In [17]:
from utils.ecr import ecr_handler
ecr = ecr_handler()

In [18]:
strRepositoryName="nemo-test-training"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerDir = "./custom-docker/"
strTag = "latest"

In [19]:
ecr.build_docker(strDockerDir, strRepositoryName, strRegionName=strRegionName, strAccountId=strAccountId)

/home/ec2-user/SageMaker/nemo-on-sagemaker/1 building-component
/home/ec2-user/SageMaker/nemo-on-sagemaker/1 building-component/custom-docker
aws ecr get-login --region 'ap-northeast-2' --registry-ids '419974056037' --no-include-email
Login Succeeded

Sending build context to Docker daemon  12.29kB

Step 1/5 : FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.13.1-gpu-py39
 ---> f0cd3f7ded0e
Step 2/5 : RUN apt -y update && apt -y install sox
 ---> Using cache
 ---> 03ea9e49dca9
Step 3/5 : RUN apt-get -y install libsox-fmt-all
 ---> Using cache
 ---> e10716b824bd
Step 4/5 : RUN pip install --no-cache-dir --upgrade pip  && pip install --no-cache-dir -U omegaconf hydra-core librosa sentencepiece youtokentome inflect sox  && pip install --no-cache-dir -U braceexpand webdataset editdistance jiwer jsonlines  && pip install --no-cache-dir -U pytorch-lightning  && pip install --no-cache-dir -qq https://github.com/pyannote/pyannote-audio/archive/develop.zip  && pip install --

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



* Push the image to ECR

In [21]:
strEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountId, strRepositoryName, strTag)

== REGISTER AN IMAGE TO ECR ==
  processing_repository_uri: 419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-training:latest
aws ecr get-login --region 'ap-northeast-2' --registry-ids '419974056037' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded

aws ecr create-repository --repository-name 'nemo-test-training'
docker tag 'nemo-test-training:latest' '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-training:latest'
docker push '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-training:latest'
== REGISTER AN IMAGE TO ECR ==


* Save image-uri to parameter store

## 4. Create inference custom docker image

* docker build

In [11]:
!pygmentize custom-docker/Dockerfile.inf

# docker build -f Dockerfile.inf -t 322537213286.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:1.13.1-gpu-py39-nemo-main .
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:1.13.1-gpu-py39

RUN pip install --no-cache-dir --upgrade pip \
 && pip install --no-cache-dir -U sagemaker nvgpu hydra-core librosa sentencepiece youtokentome inflect pyannote.audio \
 && pip install --no-cache-dir -U braceexpand webdataset editdistance jiwer jsonlines \
 && pip install --no-cache-dir -U pytorch-lightning \ 
 && pip install --no-cache-dir git+https://github.com/huggingface/transformers \ 
 && pip install --no-cache-dir git+https://github.com/NVIDIA/NeMo.git@main

COPY list_gpus.py /opt/conda/lib/python3.9/site-packages/nvgpu/list_gpus.py
WORKDIR /


### 1) AWS CLI 를 이용한 방식

In [ ]:
%%bash
strRepositoryName="nemo-test-inference"
strDockerDir="./custom-docker/"
strTag="latest"

cd ${strDockerDir}
echo $(pwd)
container_name=${strRepositoryName}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
# region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:${strTag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${container_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${container_name}" > /dev/null
fi

# # Get the login command from ECR and execute it directly
# $(aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin "763104351884.dkr.ecr.us-west-2.amazonaws.com")

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build -f Dockerfile.inf -t ${fullname} .
# docker tag ${container_name} ${fullname}

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)
docker push ${fullname}
echo ${fullname}

In [74]:
strInfEcrRepositoryUri = ' '  ## <-- 생성된 ECR의 URI를 넣어주세요.. ex) 123456789123.dkr.ecr.us-west-2.amazonaws.com/nemo-test:latest
#strInfEcrRepositoryUri = '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-inference'  ## <-- 생성된 ECR의 URI를 넣어주세요.. ex) 123456789123.dkr.ecr.us-west-2.amazonaws.com/nemo-test:latest

### 2) AWS BOTO3 SDK를 이용한 방식

In [22]:
from utils.ecr import ecr_handler
ecr = ecr_handler()

In [23]:
strInfRepositoryName="nemo-test-inference"  ## <-- 원하는 docker repostory 이름을 추가
strInfRepositoryName = strInfRepositoryName.lower()
strDockerDir = "./custom-docker/"
strTag = "latest"

In [24]:
ecr.build_docker(strDockerDir, strInfRepositoryName, strRegionName=strRegionName, strAccountId=strAccountId)

/home/ec2-user/SageMaker/nemo-on-sagemaker/1 building-component
/home/ec2-user/SageMaker/nemo-on-sagemaker/1 building-component/custom-docker
aws ecr get-login --region 'ap-northeast-2' --registry-ids '419974056037' --no-include-email
Login Succeeded

Sending build context to Docker daemon  12.29kB

Step 1/5 : FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.13.1-gpu-py39
 ---> f0cd3f7ded0e
Step 2/5 : RUN apt -y update && apt -y install sox
 ---> Using cache
 ---> 03ea9e49dca9
Step 3/5 : RUN apt-get -y install libsox-fmt-all
 ---> Using cache
 ---> e10716b824bd
Step 4/5 : RUN pip install --no-cache-dir --upgrade pip  && pip install --no-cache-dir -U omegaconf hydra-core librosa sentencepiece youtokentome inflect sox  && pip install --no-cache-dir -U braceexpand webdataset editdistance jiwer jsonlines  && pip install --no-cache-dir -U pytorch-lightning  && pip install --no-cache-dir -qq https://github.com/pyannote/pyannote-audio/archive/develop.zip  && pip install --

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [25]:
strInfEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountId, strInfRepositoryName, strTag)

== REGISTER AN IMAGE TO ECR ==
  processing_repository_uri: 419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-inference:latest
aws ecr get-login --region 'ap-northeast-2' --registry-ids '419974056037' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded

aws ecr create-repository --repository-name 'nemo-test-inference'
docker tag 'nemo-test-inference:latest' '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-inference:latest'
docker push '419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-inference:latest'
== REGISTER AN IMAGE TO ECR ==


## 5. Download & Upload dataset

In [59]:
import os
import wget

In [60]:
data_dir = "./data"

In [61]:
print("******")
os.makedirs(data_dir, exist_ok=True)
if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
    an4_url = 'https://dldata-public.s3.us-east-2.amazonaws.com/an4_sphere.tar.gz'
    an4_path = wget.download(an4_url, data_dir)
    print(f"Dataset downloaded at: {an4_path}")
else:
    print("Tarfile already exists.")
    an4_path = data_dir + '/an4_sphere.tar.gz'

******
Tarfile already exists.


In [41]:
strRepositoryName="nemo-test-inference"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerDir = "./custom-docker/"
strTag = "latest"

* updoad data to s3

### 1) AWS CLI 를 이용한 방식

In [62]:
!aws s3 sync $data_dir s3://$bucket_name/$prefix/data --quiet

### 2) AWS BOTO3 SDK를 이용한 방식

In [46]:
from utils.s3 import s3_handler

In [47]:
s3 = s3_handler()

This is a S3 handler with [None] region.


In [50]:
source_dir, target_bucket, target_dir = data_dir, pm.get_params(key=prefix+"-BUCKET"), prefix+"/data"
s3.upload_dir(source_dir, target_bucket, target_dir)

Upload:[./data] was uploaded to [s3://sm-nemo-bucket/nemo-asr/data]successfully


## 6. Upload Pretrained model

In [63]:
pretrained = os.getcwd() + '/pretrained/CTC.nemo'

pretrained_s3uri = os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "pretrained",
)

!aws s3 sync pretrained $pretrained_s3uri

## 7. CodeCommit 생성
- Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **CodeCommit**: "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess"<BR>
> **SecretsManager**: "arn:aws:iam::aws:policy/SecretsManagerReadWrite"<BR>

### 5.1 CodeCommit 관련 Credentials 생성 및 Secret Manager에 저장하기
- CodeCommit Credentials

In [64]:
user_name = 'dongjin' ## ==> IAM에서 사용자 아이디 확인합니다.
codecommit_cred = 'codecommit-cred-'+user_name
codecommit_cred

'codecommit-cred-dongjin'

In [65]:
iam_client = boto3.client('iam')

In [66]:
try:
    response = iam_client.list_service_specific_credentials(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    if len(response['ServiceSpecificCredentials']) > 0:
        response = iam_client.delete_service_specific_credential(
            UserName=user_name,
            ServiceSpecificCredentialId=response['ServiceSpecificCredentials'][-1]['ServiceSpecificCredentialId']
        )
except:
    print("Create new codecommit crendentials")
    pass
finally:
    response = iam_client.create_service_specific_credential(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    ServiceUserName = response['ServiceSpecificCredential']['ServiceUserName']
    ServicePassword = response['ServiceSpecificCredential']['ServicePassword']
print(f"ServiceUserName : {ServiceUserName} \nServicePassword : {ServicePassword}")

ServiceUserName : dongjin-at-419974056037 
ServicePassword : ldBfamxaO1suIzptEE7VyYkjVz6DQnb5EGYhrfganFs=


In [67]:
code_repository_name = 'nemo-code' ## ==> 사용할 code repository 폴더 명을 넣습니다. ex) model_code
local_code_dir = './code' ## ==> 생성한 local의 code repository 폴더 명을 넣습니다. ex) code

In [68]:
codecommit = boto3.client('codecommit')

try:
    response = codecommit.create_repository(
        repositoryName=code_repository_name,
        repositoryDescription='Data Scientists share their training code using this Repository'
    )
except:
    
    print("Repository already exists")
    response = codecommit.get_repository(
        repositoryName=code_repository_name
    )

Repository already exists


In [69]:
codecommit_repo = response['repositoryMetadata']['cloneUrlHttp']
codecommit_repo

'https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/nemo-code'

In [ ]:
!rm -rf .git/

In [ ]:
!git init
!git remote add repo_codecommit $codecommit_repo
!git checkout -b main
!git add $local_code_dir
!git commit -m "code-update"
!git push --set-upstream repo_codecommit main

## 6. [Optional] AWS Systems Manager Parameter Store 를 이용한 파라미터 저장/활용
- [AWS Systems Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html)
- Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **SSM**: "arn:aws:iam::aws:policy/AmazonSSMFullAccess"<BR>

In [70]:
from utils.ssm import parameter_store

In [71]:
pm = parameter_store(strRegionName)

In [78]:
pm.put_params(key="PREFIX", value=prefix, overwrite=True)
pm.put_params(key="-".join([prefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([prefix, "BUCKET"]), value=bucket_name, overwrite=True)
pm.put_params(key="-".join([prefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([prefix, "ACCOUNT-ID"]), value=strAccountId, overwrite=True)
pm.put_params(key="-".join([prefix, "IMAGE-URI"]), value=strEcrRepositoryUri, overwrite=True)
pm.put_params(key="-".join([prefix, "INF-IMAGE-URI"]), value=strInfEcrRepositoryUri, overwrite=True)
pm.put_params(key="-".join([prefix, "S3-DATA-PATH"]), value=f"s3://{bucket_name}/{prefix}/data", overwrite=True)
pm.put_params(key="-".join([prefix, "CODE_REPO"]), value=codecommit_repo.replace('https://',''), overwrite=True)  ## https:// 있을 경우 입력 못함
pm.put_params(key="-".join([prefix, "CODECOMMIT-USERNAME"]), value=ServiceUserName, overwrite=True, enc=True)
pm.put_params(key="-".join([prefix, "CODECOMMIT-PWD"]), value=ServicePassword, overwrite=True, enc=True)
pm.put_params(key="-".join([prefix, "PRETRAINED-WEIGHT"]), value=pretrained_s3uri, overwrite=True)
pm.put_params(key="-".join([prefix, "RETRAIN"]), value=False, overwrite=True)

'Store suceess'

In [79]:
print (f'PREFIX: {pm.get_params(key="PREFIX")}')
print (f'REGION: {pm.get_params(key="-".join([prefix, "REGION"]))}')
print (f'BUCKET: {pm.get_params(key="-".join([prefix, "BUCKET"]))}')
print (f'AGEMAKER-ROLE-ARN: {pm.get_params(key="-".join([prefix, "SAGEMAKER-ROLE-ARN"]))}')
print (f'ACCOUNT-ID: {pm.get_params(key="-".join([prefix, "ACCOUNT-ID"]))}')
print (f'IMAGE-URI: {pm.get_params(key="-".join([prefix, "IMAGE-URI"]))}')
print (f'INF-IMAGE-URI: {pm.get_params(key="-".join([prefix, "INF-IMAGE-URI"]))}')
print (f'S3-DATA-PATH: {pm.get_params(key="-".join([prefix, "S3-DATA-PATH"]))}')
print (f'CODE_REPO: {pm.get_params(key="-".join([prefix, "CODE_REPO"]))}')
print (f'CODECOMMIT-USERNAME: {pm.get_params(key="-".join([prefix, "CODECOMMIT-USERNAME"]), enc=False)}')
print (f'CODECOMMIT-PWD: {pm.get_params(key="-".join([prefix, "CODECOMMIT-PWD"]), enc=False)}')
print (f'PRETRAINED-WEIGHT: {pm.get_params(key="-".join([prefix, "PRETRAINED-WEIGHT"]))}')
print (f'PRETRAINED-WEIGHT: {pm.get_params(key="-".join([prefix, "RETRAIN"]))}')

PREFIX: nemo-asr
REGION: ap-northeast-2
BUCKET: sm-nemo-bucket
AGEMAKER-ROLE-ARN: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
ACCOUNT-ID: 419974056037
IMAGE-URI: 419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-training
INF-IMAGE-URI: 419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-inference
S3-DATA-PATH: s3://sm-nemo-bucket/nemo-asr/data
CODE_REPO: git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/nemo-code
CODECOMMIT-USERNAME: AQICAHjFzhQy64RiT2kLIfWwY+jSgA3nnCm2+lidS9TmTKjDHQHlaQMWqK/jnhQpiMRPS6zXAAAAdTBzBgkqhkiG9w0BBwagZjBkAgEAMF8GCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMea232BaCAFbhVJzRAgEQgDJRJgc2TEL0tC+qOO5MWaHpAYNqN0H6QLJ4He35g/Jn60dZrs7F1OfnpzkhL1zJwhiZEA==
CODECOMMIT-PWD: AQICAHjFzhQy64RiT2kLIfWwY+jSgA3nnCm2+lidS9TmTKjDHQEQNdpOv1gt8axi8hKaV0DzAAAAizCBiAYJKoZIhvcNAQcGoHsweQIBADB0BgkqhkiG9w0BBwEwHgYJYIZIAWUDBAEuMBEEDBiGW/C1hMNA/oFS5gIBEIBH+ogP0RlnzbWAXNe7zKuOlx0OTsLPE1n6kL01TvneLMVboXjGU3nuIjIdyU393738TMYwdMpKXA